In [365]:
import numpy as np
from collections import Counter
import nltk
nltk.download('brown')
nltk.download("stopwords")
from nltk.corpus import brown, stopwords
import re

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\alexa\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alexa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [366]:
stopwords = stopwords.words("english")

In [367]:
corpus = brown.sents()
window_size = 5
embedding_size = 10
epochs = 1000
lr = 10e-5

In [368]:
sentences = [[word for word in re.sub(r"\.|``|''",""," ".join(sentence)).lower().split() if word not in stopwords] for sentence in corpus]
T = np.sum([len(sentence) for sentence in sentences])

In [369]:
def create_vocab(sentences) :
    vocabulary = {}
    vocabulary_inverse = {}
    i = 0
    for sentence in sentences :
        for word in sentence :
            if word not in vocabulary.keys():
                vocabulary[word] = i
                vocabulary_inverse [i] = word
                i+=1
    return vocabulary, vocabulary_inverse

In [370]:
vocabulary, _ = create_vocab(sentences)
vocab_size = len(vocabulary)

In [371]:
def to_one_hot(word) :
    if word not in vocabulary.keys() : return print(f"{word} not in corpus")
    I = np.eye(vocab_size)
    return I[vocabulary[word]]

In [372]:
def generate_label(word, sentence) :
    target_index = vocabulary[word]
    window = range(target_index-window_size, target_index+window_size+1)
    words_context = sentence[window]
    context_idxs = [vocabulary[word_context] for word_context in words_context]
    y = np.asarray([1 if k in context_idxs else 0 for k in range(vocab_size)])
    return y

In [373]:
vocabulary

{'fulton': 0,
 'county': 1,
 'grand': 2,
 'jury': 3,
 'said': 4,
 'friday': 5,
 'investigation': 6,
 "atlanta's": 7,
 'recent': 8,
 'primary': 9,
 'election': 10,
 'produced': 11,
 'evidence': 12,
 'irregularities': 13,
 'took': 14,
 'place': 15,
 'term-end': 16,
 'presentments': 17,
 'city': 18,
 'executive': 19,
 'committee': 20,
 ',': 21,
 'over-all': 22,
 'charge': 23,
 'deserves': 24,
 'praise': 25,
 'thanks': 26,
 'atlanta': 27,
 'manner': 28,
 'conducted': 29,
 'september-october': 30,
 'term': 31,
 'charged': 32,
 'superior': 33,
 'court': 34,
 'judge': 35,
 'durwood': 36,
 'pye': 37,
 'investigate': 38,
 'reports': 39,
 'possible': 40,
 'hard-fought': 41,
 'mayor-nominate': 42,
 'ivan': 43,
 'allen': 44,
 'jr': 45,
 'relative': 46,
 'handful': 47,
 'received': 48,
 'considering': 49,
 'widespread': 50,
 'interest': 51,
 'number': 52,
 'voters': 53,
 'size': 54,
 'find': 55,
 'many': 56,
 "georgia's": 57,
 'registration': 58,
 'laws': 59,
 'outmoded': 60,
 'inadequate': 61,
 'o

In [374]:
def softmax(u):
    exp = np.exp(u-max(u))
    return exp/np.sum(exp)

In [375]:
W1 = np.random.randn(vocab_size, embedding_size)
W2 = np.random.randn(embedding_size,vocab_size)

In [376]:
loss = []

In [377]:
def forward(word) :
    #Forward pass
    x = to_one_hot(word)
    h = np.dot(x,W1)
    u = np.dot(h,W2)
    y_pred = softmax(u)
    return x,h,u,y_pred

In [378]:
for epoch in range(epochs) :
    L = 0
    for sentence in sentences :
        for i,word in enumerate(sentence) :
            #Forward pass
            x,h, u, y_pred = forward(word)
            window = range(max(0,i-window_size), min(i+window_size, len(sentence)))
            context_idxs = [vocabulary[sentence[k]] for k in window]
            L += -np.sum(np.log(y_pred[context_idxs]))

            #Backward pass
            y_pred[context_idxs]-=1
            dL_W2 = np.outer(h, y_pred)
            dL_h = np.dot(W2, y_pred)
            dL_W1 = np.outer(x,dL_h)

            W1 = W1 - lr*dL_W1
            W2 = W2 - lr*dL_W2
    loss.append(L)

MemoryError: Unable to allocate 18.2 GiB for an array with shape (49402, 49402) and data type float64

In [ ]:
print(loss[1],loss[-1])

1143.1858407755835 929.0479964010374


In [ ]:
class SkipGram :
    def __init__(self, corpus, window_size, embedding_size, learning_rate) :
        pass
    